In [1]:
import numpy as np
import random
import time
import operator
from collections import defaultdict
from genetic_algorithm_trap import ga

In [2]:
# Set general parameters
chromosome_length = 100
population_size = 10
trap = 1
k=4
d=2.5
tightly_linked = 0

run_dict = {}
reliable_num = 0
run_num_gen = defaultdict(list)
run_num_fitnessfunc = defaultdict(list)
run_cpu_time = defaultdict(list)

while(reliable_num < 24):
    population_size_array = np.array([])
    outcome_array = np.array([]).reshape(0,2)
    new_population_size = 10
    crossover_operator = 'UX'
    bisection_search = 0
    success_population_sizes = np.array([])
    num_generation = 0 
    num_fitnessfunc = 0
    #success_population = np.array([]).reshape(0,100)
    while(True):
        start = time.time()
        population = ga.create_starting_population(int(new_population_size), chromosome_length)
        outcome = ga.run_generation(population, crossover_operator,trap,k,d,tightly_linked)

        #num_fitnessfunc += 1
        population_size_array = np.hstack((population_size_array,new_population_size))

        if(outcome == "Fail" and new_population_size <= 2560):
            #print(f'Fail - (population size: {len(population)})')
            num_generation += 1
            if(bisection_search == 0):
                new_population_size = new_population_size*2
            else:
                midpoint_gap = abs((population_size_array[-1] - population_size_array[-2]))/2
                if(midpoint_gap < 10):
                    break
                new_population_size =  population_size_array[-1] + midpoint_gap    

        else:
            #print(len(population),outcome)
            #print(f'Success - (population size: {len(population)})')
            #num_generation += 1
            success_population_sizes = np.hstack((success_population_sizes,new_population_size))
            outcome_array = np.vstack((outcome_array,outcome))
        
            
            if(len(population_size_array)==1):
                
                break
                
            bisection_search = 1
            midpoint_gap = abs((population_size_array[-1] - population_size_array[-2]))/2
            
            if(midpoint_gap < 10):
                
                break
            new_population_size = population_size_array[-1] - midpoint_gap
        
    end = time.time()
    elapsed_time = end - start
    optimal_population_size = success_population_sizes[-1]
   

    if (str(int(optimal_population_size)) not in run_dict.keys()):
        run_dict[str(int(optimal_population_size))] = [0,0,0,0]
    
        
    run_dict[str(int(optimal_population_size))][0] += 1
    
    run_dict[str(int(optimal_population_size))][1] += outcome_array[-1][0]
    run_num_gen[str(int(optimal_population_size))].append(outcome_array[-1][0])
    
    run_dict[str(int(optimal_population_size))][2] += outcome_array[-1][1]
    run_num_fitnessfunc[str(int(optimal_population_size))].append(outcome_array[-1][1])
    
    run_dict[str(int(optimal_population_size))][3] += elapsed_time
    run_cpu_time[str(int(optimal_population_size))].append(elapsed_time)
    
    run_dict_sorted = dict(sorted(run_dict.items(), key=operator.itemgetter(1),reverse=True))
    reliable_num = run_dict_sorted[(list(run_dict_sorted.keys())[0])][0]
    

optimal_population_size = int(list(run_dict_sorted.keys())[0])
observations = run_dict_sorted[list(run_dict_sorted.keys())[0]]
averaged_observations = [i/observations[0] for i in observations[1:]]

f = open("observations.txt", "a")

f.write(f'Chromosome_length: {chromosome_length}\n')
f.write(f'Crossover operator used: {crossover_operator}\n')
if(trap==0):
    f.write(f'Fitness function: Counting Ones\n')
else:
    if(d==1):
        if(tightly_linked==1):
            f.write(f'Deceptive (tightly-linked) (k={k},d={d})\n') 
        else:
            f.write(f'Deceptive (non-tightly-linked) (k={k},d={d})\n')
    else:
        if(tightly_linked==1):
            f.write(f'Non-Deceptive (tightly-linked) (k={k},d={d})\n') 
        else:
            f.write(f'Non-Deceptive (non-tightly-linked) (k={k},d={d})\n')
f.write(f'Optimal_population_size: {optimal_population_size}\n')
f.write(f'Average number of generations for optimal_population_size: {int(averaged_observations[0])} (std: {np.std(run_num_gen[str(optimal_population_size)])})\n')
f.write(f'Average number of fitness function evaluations for optimal_population_size: {int(averaged_observations[1])} (std: {np.std(run_num_fitnessfunc[str(optimal_population_size)])})\n')
f.write(f'Average CPU time for optimal_population_size: {averaged_observations[2]} (std: {np.std(run_cpu_time[str(optimal_population_size)])})\n')
f.write(f'\n\n')
f.close()

In [3]:
run_dict_sorted #non-deceptive trap

{'40': [24, 26.0, 546.0, 5.630385398864746],
 '20': [21, 21.0, 231.0, 0.6179354190826416],
 '10': [14, 14.0, 84.0, 0.2120203971862793],
 '70': [10, 10.0, 360.0, 1.0603845119476318],
 '80': [9, 9.0, 369.0, 8.717803001403809],
 '30': [9, 9.0, 144.0, 0.40879392623901367],
 '60': [8, 12.0, 372.0, 4.787309169769287],
 '50': [7, 7.0, 182.0, 0.5329389572143555],
 '90': [6, 7.0, 322.0, 0.9413938522338867],
 '100': [5, 5.0, 255.0, 6.216620683670044],
 '110': [4, 4.0, 224.0, 0.6292660236358643],
 '170': [3, 3.0, 258.0, 0.7548799514770508],
 '150': [3, 3.0, 228.0, 0.6667604446411133],
 '130': [3, 3.0, 198.0, 0.5964818000793457],
 '210': [1, 1.0, 106.0, 0.3055243492126465],
 '180': [1, 1.0, 91.0, 3.1210508346557617],
 '160': [1, 1.0, 81.0, 3.0376229286193848]}

In [3]:
run_dict_sorted #deceptive trap tightly linked

{'40': [24, 24.0, 480.0, 0.07298612594604492],
 '20': [12, 12.0, 120.0, 0.011677026748657227],
 '60': [6, 6.0, 180.0, 0.050072669982910156],
 '10': [6, 6.0, 30.0, 0.004375934600830078],
 '110': [5, 5.0, 275.0, 0.024960756301879883],
 '30': [5, 5.0, 75.0, 0.0059854984283447266],
 '170': [4, 4.0, 340.0, 0.031018495559692383],
 '80': [4, 4.0, 160.0, 0.033843278884887695],
 '70': [4, 4.0, 140.0, 0.014371871948242188],
 '50': [4, 4.0, 100.0, 0.008040904998779297],
 '90': [3, 3.0, 135.0, 0.011062145233154297],
 '210': [1, 1.0, 105.0, 0.009004831314086914],
 '160': [1, 1.0, 80.0, 0.03390932083129883],
 '150': [1, 1.0, 75.0, 0.0059931278228759766],
 '130': [1, 1.0, 65.0, 0.006984710693359375],
 '100': [1, 1.0, 50.0, 0.013962268829345703]}

In [4]:
run_dict_sorted #counting-ones #UX

{'20': [24, 24.0, 264.0, 0.47948765754699707],
 '40': [17, 17.0, 357.0, 2.111816167831421],
 '70': [12, 12.0, 432.0, 0.866034746170044],
 '10': [11, 11.0, 66.0, 0.10509467124938965],
 '50': [9, 15.0, 390.0, 0.7617299556732178],
 '80': [9, 9.0, 369.0, 3.152785539627075],
 '30': [9, 9.0, 144.0, 0.29491424560546875],
 '100': [8, 8.0, 408.0, 5.284463167190552],
 '60': [8, 8.0, 248.0, 2.14479923248291],
 '90': [6, 8.0, 368.0, 0.8298397064208984],
 '150': [5, 5.0, 380.0, 0.9088797569274902],
 '170': [3, 3.0, 258.0, 0.595595121383667],
 '130': [3, 3.0, 198.0, 0.4000868797302246],
 '250': [1, 1.0, 126.0, 0.29288554191589355],
 '180': [1, 1.0, 91.0, 1.4502575397491455],
 '140': [1, 1.0, 71.0, 1.4940369129180908],
 '120': [1, 1.0, 61.0, 1.1260251998901367],
 '110': [1, 1.0, 56.0, 0.11165475845336914]}

In [3]:
run_dict_sorted #counting-ones

{'40': [24, 25.0, 525.0, 0.09699487686157227],
 '20': [20, 20.0, 220.0, 0.025187015533447266],
 '70': [15, 15.0, 540.0, 0.05054903030395508],
 '80': [13, 13.0, 533.0, 0.19904851913452148],
 '10': [11, 12.0, 72.0, 0.008005857467651367],
 '30': [9, 9.0, 144.0, 0.013737201690673828],
 '170': [8, 8.0, 688.0, 0.06680512428283691],
 '90': [8, 8.0, 368.0, 0.035718441009521484],
 '60': [7, 7.0, 217.0, 0.04465889930725098],
 '50': [7, 7.0, 182.0, 0.02086806297302246],
 '100': [5, 5.0, 255.0, 0.08300042152404785],
 '120': [4, 4.0, 244.0, 0.0887598991394043],
 '210': [3, 3.0, 318.0, 0.0334475040435791],
 '110': [3, 3.0, 168.0, 0.0145416259765625],
 '130': [2, 2.0, 132.0, 0.0158236026763916],
 '330': [1, 1.0, 166.0, 0.016954898834228516],
 '150': [1, 1.0, 76.0, 0.007070064544677734]}